<a href="https://colab.research.google.com/github/adrirens/MML_course/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'MyDrive/mml'       # path to your project on Drive

drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already
%cd "{PROJECT_PATH}"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/MyDrive/mml’: File exists
/content/drive/MyDrive/mml


In [3]:
GIT_USERNAME = "adrirens"  # This is a shared repository. If you want to synchronize with your own fork, well, you need to fork the github repository and replace the given name by your username.
GIT_TOKEN = "XXX"  # This token is used only if you work with your own fork. You have to generate it and put it here. Make sure to keep it confidential: it is a very sensitive information!
GIT_REPOSITORY = "MML_course"

In [4]:
# GIT_PATH = "https://"+GIT_TOKEN+"@github.com/"+GIT_USERNAME+"/"+GIT_REPOSITORY+".git"
#GIT_PATH = "https://github.com/"+GIT_USERNAME+"/"+GIT_REPOSITORY+".git"
#!git clone "{GIT_PATH}"
%cd "{GIT_REPOSITORY}"

/content/drive/MyDrive/mml/MML_course


In [5]:
!ls

base_model  figures		  image_classification_conv  model
code.ipynb  function_fitting	  logs			     README.md
dataset     image_classification  MMl_course		     solving_pde


In [6]:
!pip install torch torchvision fvcore iopath yacs tqdm numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=79b23ccd642487175f186cef4f55fdb6f44957fcba51a9d3b8a55d7e578b5812
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=5946148259f0b916c13ef85e1fdfbc9bb9cb736156347aef646cda9613740732
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


In [14]:
models = [
    "I.6.20a",
    "I.6.20",
    "I.6.20b",
    "I.8.4",
    "I.9.18",
    "I.12.2",
    "I.12.4",
    "I.12.5",
    "I.10.7",
    "I.12.11",
    "I.13.4",
    "I.13.12",
    "I.14.3",
    "I.14.4",
    "I.15.3x",
    "I.15.10",
    "I.18.4"
]

grid_sizes = [
    "5",
    "20"
]


model_types = ["KAN", "HyperKAN"]



!python image_classification_conv/train_meta.py \
  --model MetaKAN8_M \
  --n_metanets 1 \
  --optim_set double \
  --lr_h 1e-4 \
  --lr_e 1e-3 \
  --embedding_dim 1 \
  --hidden_dim 32 \
  --dataset CIFAR10 \
  --batch-size 128 \
  --epochs 50
!python image_classification_conv/train.py \
  --model KAN8 \
  --grid_size 5 \
  --spline_order 3 \
  --dataset CIFAR10 \
  --batch-size 128 \
  --epochs 50 \
  --lr 1e-3

In [17]:
import subprocess
import re
import pandas as pd
import time

def run_and_extract(cmd):
    """Exécute une commande shell et retourne les métriques extraites"""
    result = subprocess.run(cmd, capture_output=True, text=True)
    out = result.stdout

    # Extraction des infos
    num_params = re.search(r"Number of parameters:\s*([\d,]+)", out)
    test_losses = re.findall(r"test set: Average loss:\s*([\d\.Ee-]+)", out)
    train_losses = re.findall(r"train set: Average loss:\s*([\d\.Ee-]+)", out)
    time_total = re.search(r"total training time:\s*([\d\.Ee-]+)", out)
    avg_time = re.search(r"average training time per epoch:\s*([\d\.Ee-]+)", out)

    return {
        "num_params": int(num_params.group(1).replace(",", "")) if num_params else None,
        "final_train_loss": float(train_losses[-1]) if train_losses else None,
        "final_test_loss": float(test_losses[-1]) if test_losses else None,
        "total_time_sec": float(time_total.group(1)) if time_total else None,
        "avg_epoch_time": float(avg_time.group(1)) if avg_time else None,
    }

raw_results = []
for grid in grid_sizes:
  for model_name in models:
    print(f"\n🔹 Dataset: {model_name} | Grid: {grid}")
    for model_type in model_types:
        print(f"   → Training {model_type}...")

        if model_type == "KAN":
            cmd = [
                "python", "function_fitting/train.py",
                "--model", "KAN",
                "--optimizer", "lbfgs",
                "--lr", "1",
                "--dataset", model_name,
                "--layers_width", "5", "5", "5",
                "--loss", "mse",
                "--kan_bspline_grid", str(grid)
            ]
        else:  # HyperKAN
            cmd = [
                "python", "function_fitting/train_hyper.py",
                "--model", "HyperKAN",
                "--optimizer", "lbfgs",
                "--lr", "1",
                "--dataset", model_name,
                "--layers_width", "5", "5", "5",
                "--loss", "mse",
                "--embedding_dim", "1",
                "--hidden_dim", "8",
                "--kan_bspline_grid", str(grid)
            ]

        res = run_and_extract(cmd)
        res.update({
            "dataset": model_name,
            "grid": grid,
            "type": model_type
        })
        raw_results.append(res)
        time.sleep(0.5)

# 🧾 Conversion en DataFrame
df = pd.DataFrame(raw_results)

# 🪄 Transformation au format du papier
# (colonnes = G=5 KAN, G=5 HyperKAN, G=20 KAN, G=20 HyperKAN)
pivot = df.pivot_table(
    index="dataset",
    columns=["grid", "type"],
    values=["final_test_loss", "num_params"]
)

# Nettoyage du MultiIndex pour affichage plus clair
pivot.columns = [
    f"G={g} {t} {m}" for m, g, t in pivot.columns.to_flat_index()
]

# ✅ Tableau final
pivot = pivot.reset_index()
display(pivot)

# 💾 Sauvegarde CSV
pivot.to_csv("kan_comparison_table.csv", index=False)
print("\n✅ Tableau sauvegardé dans kan_comparison_table.csv")


🔹 Dataset: I.6.20a | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.6.20 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.6.20b | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.8.4 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.9.18 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.12.2 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.12.4 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.12.5 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.10.7 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.12.11 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.13.4 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.13.12 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Dataset: I.14.3 | Grid: 5
   → Training KAN...
   → Training HyperKAN...

🔹 Datase

,dataset,G=20 HyperKAN final_test_loss,G=20 KAN final_test_loss,G=5 HyperKAN final_test_loss,G=5 KAN final_test_loss,G=20 HyperKAN num_params,G=20 KAN num_params,G=5 HyperKAN num_params,G=5 KAN num_params
0,I.10.7,0.051128,0.005135,0.005615,0.004732,1272.0,3656.0,867.0,1556.0
1,I.12.11,0.304376,0.174619,0.253001,0.093857,1396.0,4176.0,961.0,1776.0
2,I.12.2,0.001809,0.003909,0.001811,0.002155,1334.0,3916.0,914.0,1666.0
3,I.12.4,0.011045,0.001275,0.002996,0.000667,1272.0,3656.0,867.0,1556.0
4,I.12.5,0.048057,0.003388,0.002972,0.001927,1210.0,3396.0,820.0,1446.0
5,I.13.12,0.013044,0.005851,0.004512,0.003597,1396.0,4176.0,961.0,1776.0
6,I.13.4,0.014097,0.004440,0.015846,0.005025,1334.0,3916.0,914.0,1666.0
7,I.14.3,0.143989,0.002667,0.012828,0.002799,1272.0,3656.0,867.0,1556.0
8,I.14.4,0.002106,0.000543,0.000818,0.001033,1210.0,3396.0,820.0,1446.0
9,I.15.10,0.195244,0.009118,0.010609,0.006371,1272.0,3656.0,867.0,1556.0



✅ Tableau sauvegardé dans kan_comparison_table.csv
